In [59]:
import networkx as nx
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import numpy as np

In [60]:
# Step 1: Create a Social Network

G = nx.Graph()
edges = [
    ('Alice', 'Bob'), 
    ('Alice', 'Charlie'), 
    ('Bob', 'Dave'), 
    ('Charlie', 'Eve'), 
    ('Dave', 'Eve'),
    ('Alice', 'Frank'),
    ('Frank', 'Grace'),
    ('Grace', 'Hannah'),
    ('Dave', 'Hannah')
]
G.add_edges_from(edges)


user_ratings = {
    'Alice': {'Movie1': 5, 'Movie2': 1},
    'Bob': {'Movie1': 4},
    'Charlie': {'Movie2': 4, 'Movie3': 5},
    'Dave': {'Movie1': 3, 'Movie3': 3},
    'Eve': {'Movie2': 4},
    'Frank': {'Movie1': 2, 'Movie2': 3},
    'Grace': {'Movie2': 5, 'Movie3': 2},
    'Hannah': {'Movie1': 4, 'Movie2': 2, 'Movie3': 5}
}


In [61]:
# Step 3: Compute Conditional Distributions

# def estimate_P_u_given_v(rating_u, rating_v):
#     if rating_v is None or rating_u is None:
#         return np.random.random()
#     return 1.0 - abs(rating_u - rating_v) / 5.0
# 
# conditional_distributions = {}
# for (u, v) in G.edges():
#     ratings_u = user_ratings.get(u, {})
#     ratings_v = user_ratings.get(v, {})
#     for movie in set(ratings_u.keys()).union(ratings_v.keys()):
#         P_u_given_v = estimate_P_u_given_v(ratings_u.get(movie), ratings_v.get(movie))
#         P_v_given_u = estimate_P_u_given_v(ratings_v.get(movie), ratings_u.get(movie))
#         conditional_distributions[(u, v, movie)] = P_u_given_v
#         conditional_distributions[(v, u, movie)] = P_v_given_u

In [62]:
# Step 4: Query Handling
def handle_query(user, movie, visited=None, depth=0):
    if visited is None:
        visited = set()
    visited.add(user)
    
    # Check if the user has rated the movie
    if movie in user_ratings.get(user, {}):
        return user_ratings[user][movie]
    
    closest_friends = list(G.neighbors(user))
    ratings = []
    
    for friend in closest_friends:
        if friend not in visited:
            if movie in user_ratings.get(friend, {}):
                weight = max(0, 1 - 0.25 * depth)
                ratings.append((user_ratings[friend][movie], weight))
            else:
                rating = handle_query(friend, movie, visited, depth + 1)
                if rating is not None:
                    weight = max(0, 1 - 0.25 * (depth + 1))
                    ratings.append((rating, weight))
    
    # Return the weighted average rating
    if ratings:
        total_weight = sum(weight for _, weight in ratings)
        weighted_average = sum(rating * weight for rating, weight in ratings) / total_weight
        return weighted_average
    
    return None


In [63]:
# Step 5: Bayesian Network

model = BayesianNetwork([
    ('Alice', 'Bob'), 
    ('Alice', 'Charlie'), 
    ('Bob', 'Dave'), 
    ('Charlie', 'Eve'), 
    ('Dave', 'Eve'),
    ('Alice', 'Frank'),
    ('Frank', 'Grace'),
    ('Grace', 'Hannah'),
    ('Dave', 'Hannah')
])

# Adding CPDs
cpd_alice = TabularCPD(variable='Alice', variable_card=2, values=[[0.5], [0.5]])
cpd_bob = TabularCPD(variable='Bob', variable_card=2, values=[[0.7, 0.2], [0.3, 0.8]], evidence=['Alice'], evidence_card=[2])
cpd_charlie = TabularCPD(variable='Charlie', variable_card=2, values=[[0.8, 0.4], [0.2, 0.6]], evidence=['Alice'], evidence_card=[2])
cpd_dave = TabularCPD(variable='Dave', variable_card=2, values=[[0.6, 0.3], [0.4, 0.7]], evidence=['Bob'], evidence_card=[2])
cpd_eve = TabularCPD(variable='Eve', variable_card=2, values=[[0.9, 0.6, 0.5, 0.3], [0.1, 0.4, 0.5, 0.7]], evidence=['Charlie', 'Dave'], evidence_card=[2, 2])
cpd_frank = TabularCPD(variable='Frank', variable_card=2, values=[[0.4, 0.1], [0.6, 0.9]], evidence=['Alice'], evidence_card=[2])
cpd_grace = TabularCPD(variable='Grace', variable_card=2, values=[[0.5, 0.7], [0.5, 0.3]], evidence=['Frank'], evidence_card=[2])
cpd_hannah = TabularCPD(variable='Hannah', variable_card=2, values=[[0.7, 0.4, 0.6, 0.2], [0.3, 0.6, 0.4, 0.8]], evidence=['Grace', 'Dave'], evidence_card=[2, 2])


model.add_cpds(cpd_alice, cpd_bob, cpd_charlie, cpd_dave, cpd_eve, cpd_frank, cpd_grace, cpd_hannah)

In [64]:
# Check model validity
model.check_model()

True

In [65]:
# Perform inference
inference = VariableElimination(model)

In [66]:
def infer_recommendation_score(user, movie):
    query_result = inference.map_query(variables=[user])
    return query_result[user]

In [68]:
# Query Recommendation Score

querying_user = 'Bob'
movie = 'Movie3'
rating = handle_query(querying_user, movie)
if rating is None:
    recommendation_score = infer_recommendation_score(querying_user, movie)
    print(f"Recommendation score for {querying_user} to watch {movie}: {recommendation_score}")
else:
    print(f"{querying_user} received rating for {movie}: {rating}")

Bob received rating for Movie3: 3.3428571428571425
